In [1]:
from pyproj import CRS, Transformer

source_crs = CRS.from_epsg(4326)  
target_crs = CRS.from_epsg(25832) 
transformer = Transformer.from_crs(source_crs, target_crs, always_xy=True)

Transformed coordinates (X, Y): (587577.7162496436, 6136866.486572812)


In [4]:
%%time
import geojson
# This file contains annotations for different types of nature phenomena (e.g. lakes, forests)
path_to_file = './naturtyper_layer.geojson'

with open(path_to_file, 'r') as f:
    gj = geojson.load(f)
print(len(gj['features']), gj.keys())

# Filter to the annotations of the lakes around Denmark
gj_features = []
for feature in gj['features']:
    if feature['properties']['Natyp_kode'] == 6: # Code for lakes is 6
        gj_features.append(feature)
print(len(gj_features))
gj_features[0]

312325 dict_keys(['type', 'name', 'crs', 'features'])
152920
CPU times: user 57.9 s, sys: 2.95 s, total: 1min
Wall time: 1min


{"geometry": {"coordinates": [[[[501332.248, 6224773.935], [501334.244, 6224779.934], [501334.244, 6224784.933], [501333.246, 6224790.932], [501327.243, 6224789.932], [501317.249, 6224783.933], [501316.243, 6224780.934], [501319.244, 6224774.935], [501324.25, 6224771.936], [501329.247, 6224770.936], [501332.248, 6224773.935]]]], "type": "MultiPolygon"}, "properties": {"Aendr_kode": 0, "Aendrbegr": "Ikke udfyldt", "Besig_dato": null, "Bruger_id": "00000000-0000-0000-0000-000000000000", "CVR_kode": 29189919, "CVR_navn": "Herning kommune", "Gl_sys_ref": null, "Journalnr": null, "Link": null, "Natyp_kode": 6, "Natyp_navn": "Sø", "Objekt_id": "0460cd7c-5353-11e2-af2b-00155d01e765", "Off_kode": 1, "Offentlig": "Synlig for alle", "Oprettet": "2006-12-31T01:00:00", "Oprindelse": "Ikke udfyldt", "Oprindkode": 0, "Sagsbeh": null, "Shape_area": 252.94599999301087, "Shape_length": 0.0, "Status": "Gældende / Vedtaget", "Statuskode": 3, "Systid_fra": "2006-12-31T01:00:00", "Systid_til": null, "Temak

In [5]:
from tqdm import tqdm
import shapely

polys = []

failed, skipped = 0, 0

# Loop over the annotations, get all the polygons
for i, feature in tqdm(enumerate(gj_features)):
    geometry = feature['geometry']
    coords = geometry['coordinates']
    try:
        poly = shapely.geometry.shape(geometry)
        if not poly.is_valid: # Skip invalid shapes
            skipped += 1
            continue
        polys.append(poly)
    except:
        pass 
print('Number of polygons:', len(polys))

152920it [00:13, 11245.04it/s]


# Danish water sample data

In [8]:
import pandas as pd
base_path = './denmark_data/water_samples_{}.xlsx'
xls_2020 = pd.ExcelFile(base_path.format(2020))
xls_2021 = pd.ExcelFile(base_path.format(2021))
xls_2022 = pd.ExcelFile(base_path.format(2022))
xls_2023 = pd.ExcelFile(base_path.format(2023))

In [9]:
df_2020 = pd.read_excel(xls_2020, 'Opdeling')

# There's a few sheets with lakes info: 'Sø' is another one which seems to include more?
df_2021 = pd.read_excel(xls_2021, 'Vandprøver 2021', header = 1)
df_2022 = pd.read_excel(xls_2022, 'Vandprøver 2022', header = 1)
df_2023 = pd.read_excel(xls_2023, 'Master', header = 1)

# Gather the dataframes together for easier access
dfs = [df_2020, df_2021, df_2022, df_2023]

In [13]:
# Obtain the longitudes (x) and latitudes (y) of the lakes from which we have water samples
latitude_str = 'Breddegrad'	
longitude_str = 'Længdegrad'
latitudes = []
longitudes = []
for df in dfs:
    latitudes_df = df[latitude_str]
    longitudes_df = df[longitude_str]
    latitudes.extend(latitudes_df.tolist())
    longitudes.extend(longitudes_df.tolist())


coordinates = [transformer.transform(long, lat) for long, lat in zip(longitudes, latitudes)]
# x, y = transformer.transform(longitude, latitude)
# coordinates = zip(longitudes, latitudes)

# Connect the location of water samples to lakes

In [ ]:
from shapely.geometry import Point, Polygon
import geopandas as gpd

# Convert x, y coordinates to Point objects
point_geometries = [Point(x, y) for x, y in points]

# Add the polygons to a GeoDF so we can easily compute distances
gdf_polygons = gpd.GeoDataFrame({'geometry': polys})

# List to store the index of the closest polygon (one for each point)
closest_polygon_idx = []

# Calculate the closest polygon for each point, and store the index of that polygon
for point in tqdm(point_geometries):
    distances = gdf_polygons.distance(point) # Calculate the distance from the point to each polygon
    closest_idx = distances.idxmin() # The index of the polygon that's closest to our point
    closest_polygon_idx.append(closest_idx)

# Print the results
for i, point in enumerate(points):
    print(f"Coordinate {point} is closest to polygon number {closest_polygon_idx[i]}")